In [135]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

%opts Layout [sublabel_format='' aspect_weight=1 figure_size=(100) vspace=0.4]
%output size = 150

randn = np.random.randn

alphas = np.linspace(0, 1, 1000)

dims = SimpleNamespace(E=holoviews.Dimension(r'$E$'),
                       alpha=holoviews.Dimension(r'$\alpha$'),
                       Q=holoviews.Dimension(r'$Q$'),
                       Q_BdG=holoviews.Dimension(r'$Q_{BdG}$'))

def make_random_real_ham(N):
    H = randn(N, N)
    H += H.T
    return H / 2


def make_cons_ham(N):
    H = np.kron(pauli.s0, randn(N, N)) + np.kron(pauli.sz, randn(N, N))
    H += H.T.conj()
    return H / 2


def make_random_ham(N):
    H = randn(N, N) + 1j * randn(N, N)
    H += H.T.conj()
    return H / 2


def make_random_symplectic_ham(N):
    if N % 2:
        raise ValueError('Matrix dimension should be a multiple of 2')
    sy = np.kron(np.eye(N // 2), np.array([[0, -1j], [1j, 0]]))
    h = randn(N, N) + 1j * randn(N, N)
    h += h.T.conj()
    Th = sy @ h.conj() @ sy
    return (h + Th) / 4


def make_chiral_ham(N):
    temp1 = randn(N, N) + 1j * randn(N, N)
    temp2 = randn(N, N) + 1j * randn(N, N)    
    H = np.kron(pauli.sx, temp1) + np.kron(pauli.sy, temp2)
    H += H.T.conj()
    return H / 2


def make_BdG_ham(N):
    # This is antisymmetric basis
    H = 1j * randn(2*N, 2*N)
    H += H.T.conj()
    return H / 2


def energies(alpha, H0, H1):
    H = (1 - alpha) * H0 + alpha * H1
    return np.linalg.eigvalsh(H)


def find_spectrum(alphas, H0, H1):
    spectrum = [energies(a, H0, H1) for a in alphas]
    return np.array(spectrum)


def find_Q(spectrum):
    """Finds the number of bands that are under zero energy.

    Parameters:
    -----------
    spectrum : numpy array
        Array that contains the energies levels for every alpha.

    Returns:
    --------
    Q : list
        Number of bands under zero energy.
    """
    return [len(s[s<0]) for s in spectrum]


def plot_hamiltonian_spectrum(alphas, spectrum, E_range=(-4, 4)):
    """Function that plots a spectrum for a range of alphas.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    spectrum : numpy array
        Array that contains the energies levels for every alpha.
    E_range : tuple
        The upper and lower limit of the y-dimension.

    Returns:
    --------
    plot : holoviews.Path object
        Plot of alphas vs. spectrum.
    """
    E_min, E_max = E_range
    plot = holoviews.Path((alphas, spectrum), kdims=[dims.alpha, dims.E])[:, E_min:E_max] * holoviews.HLine(0)
    return plot(plot={'xticks':[0, 0.5, 1], 'yticks':[E_min, 0, E_max]})


def plot_Q(alphas, Q, Q_range, Q_dim=dims.Q):
    """Function that plots value of Q for a range of alphas.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    Q : numpy array
        Vector that contains the value of Q for every alpha.
    Q_range : tuple
        The upper and lower limit of the y-dimension.

    Returns:
    --------
    plot : holoviews.Path object
        Plot of alphas vs. Q.
    """
    Q_min, Q_max = Q_range
    Q_mid = (Q_max + Q_min) / 2
    plot = holoviews.Area((alphas, Q), kdims=[dims.alpha], vdims=[Q_dim])[:, Q_min:Q_max](style={'alpha': 0.4})
    plot *= holoviews.Curve((alphas, Q), kdims=[dims.alpha], vdims=[Q_dim])[:, Q_min:Q_max]
    return plot(plot={'xticks':[0, 0.5, 1], 'yticks':[Q_min, Q_mid, Q_max], 'aspect':4})

Populated the namespace with:
np, matplotlib, kwant, holoviews, init_notebook, SimpleNamespace, pprint_matrix, scientific_number, pretty_fmt_complex, plt, pf, display_html
from code/edx_components:
MoocVideo, PreprintReference, MoocDiscussion, MoocCheckboxesAssessment, MoocMultipleChoiceAssessment, MoocPeerAssessment, MoocSelfAssessment
from code/functions:
spectrum, hamiltonian_array, h_k, pauli
Using kwant 1.3.2 and holoviews 1.9.5
Executed on 2018-03-15 at 23:21:02.576503.


# Simulations: what about other symmetries

So you've made it through the content of the first week. Congratulations!

Now let's get our hands dirty. 

Let's begin by grabbing the notebooks of this week and the extra code we use to run these notebooks over [here](http://tiny.cc/topocm_smc). (Click the [i] button to the left of the folder that you want to copy.)

You need to copy the `code` folder and the `w1_topointro` folder. Let's look into what's inside.


#### First task: combination of particle-hole and time-reversal symmetries

Look at the notebook about topology of zero-dimensional systems, and see how we generate Hamiltonians with a spinful time-reversal symmetry

$$
H = \sigma_y H^* \sigma_y.
$$

Now try to add this time reversal symmetry to a Hamiltonian which also has particle-hole symmetry. It is easiest to do in the basis where particle-hole symmetry has the form $H = -H^*$.
What do you think will happen? What will the extra symmetry do to the topological invariant?
Test your guess by plotting the spectrum and calculating the Pfaffian invariant.


#### Second task: Su-Schrieffer-Heeger (SSH) model

Similar to the Kitaev chain, the SSH model is simple a one-dimensional model where you can see all the essential aspects of topological systems. Unlike the Kitaev chain it does correspond to a physical system: electrons in a polyacetylene chain.

Here's such a chain:

![](figures/polyacetylene.png)

Due to the dimerization of the chain the unit cell has two atoms and the hoppings have alternating strengths $t_1$ and $t_2$, so that the Hamiltonian is
$$H = \sum_{n=1}^N t_1 \left|2n-1\right\rangle\left\langle 2n\right|+t_2 \left|2n\right\rangle \left\langle 2n+1\right| + \textrm{h.c}$$

We can choose to start a unit cell from an even-numbered site, so $t_1$ becomes intra-cell hopping and $t_2$ inter-cell hopping.


Now get the notebook with the Kitaev chain and transform a Kitaev chain into an SSH chain.

Now repeat the calculations we've done with Majoranas using SSH chain. Keep $t_1 = 1$ and vary $t_2$.
You should see something very similar to what you saw with the Kitaev chain.

As you can guess, this is because the chain is topological.
Think for a moment: what kind of symmetry protects the states at the edges of the chain.
*(Hint: you did encounter this symmetry in our course.)*

The particle-hole symmetry, is a consequence of a mathematical transformation, and cannot be broken.
The symmetry protecting the SSH chain, however, can be broken.
Test your guess about the protecting symmetry by adding to your chain a term which breaks this symmetry and checking what it does to the spectrum of a finite chain and to its dispersion (especially as chain goes through a phase transition).

# My simulations

## First task

Reasoning:
1. I choose fermion parity as topological invariant, because the system involves particle-hole symmetry. 
2. When varying $\alpha$, around some points the eigen-energy of eigenstates (with odd numbers of degeneracy) changes from $\gtrsim 0$ to $\lesssim 0$, fermion parity changes.
3. If I add TRS, every level is doublely degenerate. One crossing means that the energy of two degenerate eigenstates changes from $\gtrsim 0$ to $\lesssim 0$. If this crossing really happens, fermion parity would not change.
4. Because fermion parity do not change, even if the crossing happens, there exists another way to deforming Hamiltonian which transforms $H_0$ into $H_1$ without crossing. 

Conclusion:
5. The crossing cannot be protected by PHS + TRS.
6. So PHS + TRS makes fermion parity invariant and trivial. Fermion parity cannot be changed by varying the parameters of Hamiltonian.

$\diamondsuit$ Anoter way to see this: because of TRS(doublely degenerate), $H_0$ and $H_1$ (both are gapped)must have even numbers of fermions. Their topological quantity "fermion parity" are the same and trivial.

Below, graphs on the right describe a PHS+TRS system. Graphs on the left describe a PHS system.

\\
\\

Some algebra :

Conventions : $\tau_i $ act on particle-hole space. $\sigma_i$ act on spin space.

In "normal" basis  $C = (c_1, \dots, c_n, c^\dagger_1, \dots, c^\dagger_n)^T$, PHS is $\tau_x H \tau_x = - H^*$, TRS is $\sigma_y H \sigma_y = H^*$.

Matrix $H$ which only satisfies PHS (it can be non-Hermitian) must looks like :
$$ H = A \otimes \tau_z + B_1 \otimes \tau_x + B_2 \otimes \tau_y + B_3 \otimes I $$
where $A_1 \in span\{ I, i\sigma_y \}. B_1, B_2, B_3 \in span\{ iI, i\sigma_x, \sigma_y, i\sigma_z \}$.

When the additional symmtry, TRS, is imposed on $H$, it still has the same form : 
$$ H = A' \otimes \tau_z + B'_1 \otimes \tau_x + B'_2 \otimes \tau_y + B'_3 \otimes I $$
but with less possible combinations:  $A'_1 \in span\{ I, i\sigma_y \}. B'_1, B'_2, B'_3 \in span\{ i\sigma_x, i\sigma_z \}$.

Another restriction is Hermicity.

[Example1] One mode with PHS + TRS (specified by 2 parameters)
$$
H = a I \otimes \tau_z + b \sigma_y \otimes \tau_z
=
\begin{pmatrix}
  \begin{array}{@{}cc|cc@{}}
        a &  0   &    0 &  b \\
        0 &  a   &   -b &  0 \\\hline       
        0 & -b   &   -a &  0 \\
        b &  0   &    0 & -a
  \end{array}
\end{pmatrix} . \quad  a, b \in {\rm I\!R}
$$

[Example2] Two modes, 0 and 1, with PHS + TRS (specified by 12 parameters)

$$
H = (aI+id\sigma_y) \otimes \tau_z + (ib\sigma_z + ic\sigma_x) \otimes I + (ia'\sigma_z + ib'\sigma_x) \otimes \tau_x + (-ic'\sigma_y+ d' I) \otimes i\tau_y= \
\begin{pmatrix}
  \begin{array}{@{}cc|cc@{}}
        a_{00}I    &     a_{01}I+id_{01}\sigma_y+ib_{01}\sigma_z+ic_{01}\sigma_x    &  -ic'_{00}I & a'_{01}i\sigma_z + b'_{01}i\sigma_x -ic'_{01}\sigma_y + d'_{01} I \\
          &  a_{11}I   & -a'_{01}i\sigma_z - b'_{01}i\sigma_x -ic'_{01}\sigma_y - d'_{01} I  &  -ic'_{11}I \\\hline       
          &            &   -a_{00}I &   -a_{01}I-id_{01}\sigma_y+ib_{01}\sigma_z+ic_{01}\sigma_x    \\
          &            &           & -a_{11}I
  \end{array}
\end{pmatrix} \\ a, b, \dots, c', d' \in {\rm I\!R}
$$


In "anti-symmetric" basis,
$$C' =
  \begin{pmatrix}
       c'_1 \\
       c'_2 \\
       \vdots \\
       c'_n  \\
       c'^\dagger_1 \\
       c'^\dagger_2 \\
       \vdots \\
       c'^\dagger_n
     \end{pmatrix}
     = UC =  U  
  \begin{pmatrix}
       c_1 \\
       c_2 \\
       \vdots \\
       c_n  \\
       c^\dagger_1 \\
       c^\dagger_2 \\
       \vdots \\
       c^\dagger_n
     \end{pmatrix}
$$

The new WF becomes $u'$ which $ \sum u'_i = U_{ij} u_j $, such that the state would not change ( $ \sum u_i c_i = \sum u'_i c'_i $ )

The coefficient matrix $H$ in this basis becomes $ H' = UHU^\dagger $ such that the Hamiltonian would not change ( $c^\dagger H c = c'^\dagger H' c'$ )

For BdG Hamiltonian $ H_\textrm{BdG} = \begin{pmatrix} H & \Delta \\ -\Delta^* & -H^* \end{pmatrix} $ , 
$U = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ i &  -i \end{pmatrix}.$
For sublattice Hamiltonian, $U = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & -i \\ 1 &  i \end{pmatrix}.$ Both act on P-H basis.

PHS is $H = - H^*$, TRS is $\sigma_y H \sigma_y = H^*$. Any Hermitian matrix $H$ satisfying PHS must be purely imaginary and anti-symmetric, as the demonstration below.


In [136]:
holoviews.plotting.mpl.MPLPlot.fig_rcparams['text.usetex'] = False
def pureimg_antisym_ham_add_TRsym(h,N):  
    # spin1/2 system, smallest 2x2 block for spin-up & -down state
    sy = np.kron(np.eye(N // 2), np.array([[0, -1j], [1j, 0]]))
    Th = sy @ h.conj() @ sy
    return (h + Th) / 2

modes = 3
np.random.seed(5)  # can demostrate seed=4,5,7  energy repels around 0
H0 = make_BdG_ham(modes*2)
H0_TRS = pureimg_antisym_ham_add_TRsym(H0,modes*4)
pprint_matrix(H0_TRS)

H1 = make_BdG_ham(modes*2)
H1_TRS = pureimg_antisym_ham_add_TRsym(H1,modes*4)

spectrum_TRS = find_spectrum(alphas, H0_TRS, H1_TRS)
spectrum = find_spectrum(alphas, H0, H1)

#%%opts Overlay [yticks=[-1, 0, 1]]

def find_pfaffian(alphas, H0, H1):
    """Function caculates the Pfaffian for a Hamiltonian.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    H0 : numpy array
        Hamiltonian, same size as H1.
    H1 : numpy array
        Hamiltonian, same size as H0.
        
    Returns:
    --------
    pfaffians : numpy array
        Pfaffians for each alpha.
    """
    def H(alpha):
        return (1 - alpha) * H0 + alpha * H1
    pfaffians = [np.sign(np.real(pf.pfaffian(1j*H(a)))) for a in alphas]
    return np.array(pfaffians)

pfaffian_TRS = find_pfaffian(alphas, H0_TRS, H1_TRS)
pfaffian = find_pfaffian(alphas, H0, H1)

(plot_hamiltonian_spectrum(alphas, spectrum, [-1.5, 1.5]) + plot_hamiltonian_spectrum(alphas, spectrum_TRS, [-1.5, 1.5])+ plot_Q(alphas, pfaffian, [-1.5, 1.5], dims.Q_BdG) + plot_Q(alphas, pfaffian_TRS, [-1.5, 1.5], dims.Q_BdG) ).cols(2)

:Layout
   .Overlay.I   :Overlay
      .Path.I  :Path   [$\alpha$,$E$]
      .HLine.I :HLine   [x,y]
   .Overlay.II  :Overlay
      .Path.I  :Path   [$\alpha$,$E$]
      .HLine.I :HLine   [x,y]
   .Overlay.III :Overlay
      .Area.I  :Area   [$\alpha$]   ($Q_{BdG}$)
      .Curve.I :Curve   [$\alpha$]   ($Q_{BdG}$)
   .Overlay.IV  :Overlay
      .Area.I  :Area   [$\alpha$]   ($Q_{BdG}$)
      .Curve.I :Curve   [$\alpha$]   ($Q_{BdG}$)

## Second Task

I made a small change. Here the Hamiltonian in my code is : 
$$H_{ssh} = t_1 \left|2N-1\right\rangle\left\langle 2N\right| + 
\sum_{n=1}^{N-1} t_1 \left|2n-1\right\rangle\left\langle 2n\right|+t_2 \left|2n\right\rangle \left\langle 2n+1\right| + \textrm{h.c}$$
Number of site is $2N$. They are divided into $N$ cells.

This system has sublattice symmetry. The sites with odd index only couple to the sites with even index. If I write the WF as $\psi = (c_1, c_3, \dots, c_{2N-1}, c_2, c_4, \dots, c_{2N})^T$, with upper half is odd sublattice and lower half is even lattice, then the Hamiltonian in this basis looks like

$$
H_{ssh} =
\begin{pmatrix}
0 & H_{AB} \\
H_{AB}^\dagger & 0
\end{pmatrix}
=
\begin{pmatrix}
  \begin{array}{@{}ccccc|ccccc@{}}
    0 & 0 & 0 &        &                &    t_1 &   0   &   0    &             &  \\
    0 & 0 & 0 &        &                &  t_2^* & t_1   &   0    &             &  \\
    0 & 0 & 0 & \ddots &                &      0 & t_2^* &  t_1   &    \ddots  &  \\
      &   & \ddots  & \ddots & 0              &        &       & \ddots & \ddots      &   0 \\
      &   &   &   0    & 0              &        &       &  &    t_2^*    &  t_1    \\\hline       
t_1^* & t_2 &   0   &         &         &    0   &   0   &   0    &             & \\
    0 & t_1^* & t_2 &         &         &    0   &   0   &   0    &             & \\
    0 &   0 & t_1^* & \ddots  &         &    0   &   0   &   0    &   \ddots    & \\
      &     & \ddots& \ddots  &  t_2^*  &        &       & \ddots &   \ddots    &   0 \\
      &     &       &    0    &  t_1    &        &       &        &      0      &   0
  \end{array}
\end{pmatrix}
$$

Sublattice symmetry protects the states at the edges of the chain. 
Topological phase : $  t_2/t_1 < -1 $ or $  t_2/t_1 >  1 $
Trival phase : $ -1 < t_2/t_1 < 1 $

I put next nearest neighbor hopping (e.g. $<4|H_{ssh}|2>$) to break sublattice symmtry. The spectrum become not symmtric to E=0.


Note : 
To computing pfaffian at k=0 and k=-$\pi$, I need to make $H_{ssh}$  antisymmetric. If $H_{AB}$ is real, then :
$$
U = \frac{1}{\sqrt{2}} 
\begin{pmatrix}
1 & -i \\
1 &  i
\end{pmatrix}.
$$

$$ H_{antisym} = U H_{ssh}  U^\dagger  $$



In [1]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

Populated the namespace with:
np, matplotlib, kwant, holoviews, init_notebook, SimpleNamespace, pprint_matrix, scientific_number, pretty_fmt_complex, plt, pf, display_html
from code/edx_components:
MoocVideo, PreprintReference, MoocDiscussion, MoocCheckboxesAssessment, MoocMultipleChoiceAssessment, MoocPeerAssessment, MoocSelfAssessment
from code/functions:
spectrum, hamiltonian_array, h_k, pauli
Using kwant 1.3.2 and holoviews 1.9.5
Executed on 2018-03-15 at 14:14:21.333375.


In [128]:
holoviews.plotting.mpl.MPLPlot.fig_rcparams['text.usetex'] = False
%output size=150
dims = SimpleNamespace(E_t=holoviews.Dimension(r'$E/t_1$'),
                       mu_t=holoviews.Dimension(r'${t_2}/{t_1}$'),
                       lambda_=holoviews.Dimension(r'$\lambda$'),
                       x=holoviews.Dimension(r'$x$'),
                       k=holoviews.Dimension(r'$k$'),
                       amplitude=holoviews.Dimension(r'$|u|^2$'))

holoviews.core.dimension.title_format = ''

def ssh_chain(L=None, breaking=False):
    lat = kwant.lattice.chain()
    if L is None:
        syst = kwant.Builder(kwant.TranslationalSymmetry((-1,)))
        L = 3
    else:
        if L%2 != 1:
            raise ValueError('Length of the chain should be an odd number')
        else:
            syst = kwant.Builder()
    
    # transformation to antisymmetric basis
    U = np.array([[1.0, -1j], [1.0, 1j]]) / np.sqrt(2)
    
    
    def hop1(site1, site2, p):
        return p.t1
    def hop2(site1, site2, p):
        return p.t2
    
    #==============#
    N = (L-1)//2
    
    def onsite(onsite, p): 
        return p.t1 * U @ (pauli.sx ) @ U.T.conj()#* pauli.sz   #return - p.mu * U @ pauli.sz @ U.T.conj()
    
    for x in range(N):
        syst[lat(x)] = onsite

    def hop(site1, site2, p):
        return ( p.t2 * U @(  pauli.sx - 1j*pauli.sy )/2 @ U.T.conj() +
                + 1.2 * U @       pauli.s0               @ U.T.conj()   )
    
    syst[kwant.HoppingKind((1,), lat)] = hop
    
    #==============#
    '''
    for j in range(L):
        syst[lat(j)] = 0
    
    N = (L-1)//2
    for j in range(N):
        syst[lat(2*j), lat(2*j + 1)] = hop1
        syst[lat(2*j+1), lat(2*j + 2)] = hop2
    '''    
    #for j in range(N-2):
    #    syst[lat(2*j), lat(2*j + 2)] = -0.5
        
   # for j in range(N-1-2):
    #    syst[lat(2*j+1), lat(2*j + 3)] = -0.5
 
    
  
    return syst

def plot_wf(syst, wf1, lstyle='-', lcolor='b'):
    xs = np.array([i.pos[0] for i in syst.sites])
    indx = np.argsort(xs)
    wf_sq =  np.linalg.norm(wf1.reshape(-1, 2), axis=1) #np.real(wf1*wf1.conj()) #wf_sq = np.linalg.norm(wf1.reshape(-1, 2), axis=1)**2 + np.linalg.norm(wf2.reshape(-1, 2), axis=1)**2
    wf_sq = np.real(wf_sq*wf_sq.conj())
    plot = holoviews.Path((xs[indx], wf_sq[indx]), kdims=[dims.x, dims.amplitude])
    return plot(style={'linestyle': lstyle, 'color': lcolor},
                plot={'yticks': 0, 'xticks': list(range(0, len(xs), 10))})

def plot_pwave(L, t1x, t2x):
    # At mu=0 the first exited state is not well defined due to the massive degeneracy.??
    # That is why we add a small offset to mu.??
    syst = ssh_chain(L).finalized()
    p = SimpleNamespace(t1=t1x, t2=t2x) # p = SimpleNamespace(t=t, delta=delta, mu=mu+1e-4)
    ham = syst.hamiltonian_submatrix(args=[p])
    ev, evec = np.linalg.eigh(ham)
    return (plot_wf(syst, evec[:, (L-1)//2])  *plot_wf(syst, evec[:, (L+1)//2], lstyle='--', lcolor='r'))
    
L=15
t1=1
syst = ssh_chain(L)
t2s = np.arange(-5, 5, 0.4)+1e-4           

p = SimpleNamespace(t1=1.0, t2=t2s)  

(spectrum(syst, p, xticks=range(-2, 3), yticks=range(-3, 4), xdim=dims.mu_t, ydim=dims.E_t, ylims=(-3, 3))
 * holoviews.HoloMap({t2: holoviews.VLine(t2) for t2 in t2s}, kdims=[dims.mu_t]) + 
 holoviews.HoloMap({t2: plot_pwave(L, t1, t2) for t2 in t2s}, kdims=[dims.mu_t]))



:Layout
   .HoloMap.I  :HoloMap   [${t_2}/{t_1}$]
      :Overlay
         .Path.I  :Path   [${t_2}/{t_1}$,$E/t_1$]
         .VLine.I :VLine   [x,y]
   .HoloMap.II :HoloMap   [${t_2}/{t_1}$]
      :Overlay
         .Path.I  :Path   [$x$,$|u|^2$]
         .Path.II :Path   [$x$,$|u|^2$]

In [133]:
def bandstructure(t2, t1=1, delta=1, Dirac_cone="Hide", show_pf=False):
    syst = ssh_chain(None)
    p = SimpleNamespace(t2=t2+1e-4, delta=delta, t1=t1)
    plot = holoviews.Overlay([spectrum(syst, p, ydim="$E/T$", xdim='$k$')][-4:4])
    h_1 = h_k(syst, p, 0)
    h_2 = h_k(syst, p, np.pi)
    t2_t1 = round(t2/t1,1)
    title = "$t_2 = {t} t_1$, topological ".format(t=t2_t1)
    
    if abs((h_1+h_1.T).max()) < 1e-14 and abs((h_2+h_2.T).max()) < 1e-14 :
        pfaffians = [find_pfaffian(h_1), find_pfaffian(h_2)] 
        if show_pf:
            signs = [('>' if pf > 0 else '<') for pf in pfaffians]
            title = "$t_2 = {t} t_1$, Pf$(iH_{{k=0}}) {sign1} 0$, Pf$(iH_{{k=\pi}}) {sign2} 0$"
            title = title.format(t=t2_t1, sign1=signs[0], sign2=signs[1])
            plot *= holoviews.VLine(0) * holoviews.VLine(-np.pi)
        else:
            if pfaffians[0] * pfaffians[1] < 0:
                title = "$t_2 = {t} t_1$, topological ".format(t=t2_t1)
            else:
                title = "$t_2 = {t} t_1$, trivial ".format(t=t2_t1)
    else:
        title = "$t_2 = {t} t_1$, unknown pfaffians ".format(t=t2_t1)
    #if Dirac_cone == "Show":
    #    ks = np.linspace(-np.pi, np.pi)
     #   ec = np.sqrt((mu + 2 * t)**2 + 4.0 * (delta * ks)**2)
     #   plot *= holoviews.Path((ks, ec), kdims=[dims.k, dims.E_t])(style={'linestyle':'--', 'color':'r'})
     #   plot *= holoviews.Path((ks, -ec), kdims=[dims.k, dims.E_t])(style={'linestyle':'--', 'color':'r'})
    return plot.relabel(title)

def find_pfaffian(H):
    return np.sign(np.real(pf.pfaffian(1j*H)))


t2s = np.arange(-2, 2, 0.2)+1e-4  # small offset for an unknow problem at t2 = +-1

plots = {t2: bandstructure(t2, show_pf=False)  
         for t2 in t2s}

holoviews.HoloMap(plots, kdims=[dims.mu_t])

:HoloMap   [${t_2}/{t_1}$]
   :Overlay
      .Path.I :Path   [$k$,$E/T$]

In [ ]:
MoocSelfAssessment()

**Now share your results:**

In [ ]:
MoocDiscussion('Labs', 'Toy models simulations')

# Review assignment

For the first week we have these papers:

In [ ]:
display_html(PreprintReference('1103.0780', 
  description="Topological classification is not always applied to Hamiltonians. "
              "Figure out what is the topological quantity in open systems. "
              "See this idea also applied in <a href=http://arxiv.org/abs/1405.6896>"
              "http://arxiv.org/abs/1405.6896</a>"))
display_html(PreprintReference('1305.2924', description="This is a study of statistical properties "
                                       "of topological transitions."))
display_html(PreprintReference('1111.6600', description="A toy model may still be useful in practice."))

### Bonus: Find your own paper to review!

Do you know of another paper that fits into the topics of this week, and you think is good?
Then you can get bonus points by reviewing that paper instead!

In [ ]:
MoocPeerAssessment()

**Do you have questions about what you read? Would you like to suggest other papers? Tell us:**

In [ ]:
MoocDiscussion("Reviews", "Toy models")